<a href="https://colab.research.google.com/github/MayerT1/LiDAR/blob/main/Working_Merge_Laz_veg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/PhD_Main_Folder/Geo_Data

/content/drive/MyDrive/PhD_Main_Folder/Geo_Data/GEDI_DEV_GUAN


In [ ]:

# ################# Transfomr the UTM to DDM

# import pandas as pd
# from pyproj import Transformer


# # df3
# lidar_gdf

# # # Load your CSV
# # df = pd.read_csv("your_file.csv")  # replace with your actual filename

# # Initialize the transformer (e.g., UTM Zone 33N to WGS84)
# transformer = Transformer.from_crs("EPSG:32619", "EPSG:4326", always_xy=True)

# # Define a function to convert each row
# def convert_coords(row):
#     lon, lat = transformer.transform(row['Easting'], row['Northing'])
#     return pd.Series({'Latitude_lidar': lat, 'Longitude_lidar': lon})

# # Apply the function to each row
# lidar_gdf[['Latitude_lidar', 'Longitude_lidar']] = lidar_gdf.apply(convert_coords, axis=1)

# # # Save the new DataFrame to a new CSV
# # df.to_csv("converted_coordinates.csv", index=False)

# # print("Conversion complete! Saved to converted_coordinates.csv")
# lidar_gdf

# # # Save the new DataFrame to a new CSV
# lidar_gdf.to_csv("lidar_gdfconverted_coordinat.csv", index=False)

In [ ]:
import os
import laspy
import pandas as pd
import geopandas as gpd
from scipy.stats import spearmanr
from shapely.geometry import Point



# Define file paths
laz_path = "/content/drive/MyDrive/PhD_Main_Folder/Geo_Data/Neon_PR_Data/GUAN/2018/laz/" # Update this path
# laz_file = os.path.join(laz_path, "NEON_D04_GUAN_DP1_725000_1985000_classified_point_cloud_colorized.laz")
laz_file = os.path.join(laz_path, "NEON_D04_GUAN_DP1_727000_1986000_classified_point_cloud_colorized.laz")

veg_file = "/content/drive/MyDrive/PhD_Main_Folder/Geo_Data/Neon_PR_Data/GUAN/Veg_Field_Data/veg.csv"  # The uploaded vegetation data file

# # Load LiDAR data
# las = laspy.read(laz_file)
# # lidar_df = pd.DataFrame({
# #     "Easting": las.x,
# #     "Northing": las.y,
# #     "Elevation": las.z,
# #     "Intensity": las.intensity
# # })
# # #lidar_df.head()

# lidar_df = pd.DataFrame({
#     "Easting": np.array(las.x),        # Convert ScaledArrayView to NumPy array
#     "Northing": np.array(las.y),
#     "Lidar_z": np.array(las.z),
#     "Intensity": np.array(las.intensity)
# })
# #lidar_df.head()


#
#
#
# Update the transformed the from local
#
#
#

#https://drive.google.com/drive/folders/1JeKEirJH6d6ZrRAoYaLHlV_6L8ofMXxt
lidar_df = "/content/drive/MyDrive/PhD_Main_Folder/Geo_Data/converted_coordinates.csv"  # The uploaded vegetation data file
# Load vegetation data
lidar_df = pd.read_csv(lidar_df)

# Load vegetation data
veg_df = pd.read_csv(veg_file)

# # Convert LiDAR and Veg data to GeoDataFrames for spatial joining
# lidar_gdf = gpd.GeoDataFrame(lidar_df, geometry=gpd.points_from_xy(lidar_df.Easting, lidar_df.Northing))
# veg_gdf = gpd.GeoDataFrame(veg_df, geometry=gpd.points_from_xy(veg_df["adjEasting"], veg_df["adjNorthing"]))

# Convert LiDAR and Veg data to GeoDataFrames for spatial joining
lidar_gdf = gpd.GeoDataFrame(lidar_df, geometry=gpd.points_from_xy(lidar_df["Longitude"], lidar_df["Latitude"]]))
veg_gdf = gpd.GeoDataFrame(veg_df, geometry=gpd.points_from_xy(veg_df["adjDecimalLongitude"], veg_df["adjDecimalLatitude"]))


# Perform a spatial join to find overlapping LiDAR and vegetation data
merged_gdf = gpd.sjoin_nearest(veg_gdf, lidar_gdf, how="inner")  # Adjust distance threshold # max_distance=50000
# print(merged_gdf)


# Compute correlation between LiDAR elevation and vegetation height
if "height" in merged_gdf.columns:  # Ensure 'height' exists in the Veg data
    correlation, p_value = spearmanr(merged_gdf["Lidar_z"], merged_gdf["height"])
    print(f"Spearman Correlation: {correlation}, p-value: {p_value}")
else:
    print("No 'height' column found in vegetation data. Check column names.")

# Save merged data for further analysis
merged_gdf.to_csv("/content/drive/MyDrive/PhD_Main_Folder/Geo_Data/Neon_PR_Data/GUAN/Veg_Field_Data/lidar_veg_correlation.csv", index=False)
print("Merged dataset saved as 'lidar_veg_correlation.csv'.")